# Doctor List

In [ ]:
import pickle
import pandas as pd

In [ ]:
path = 'Data/.p'

MD_Doc = pd.read_pickle(path)

In [ ]:
path = 'Data/MD_DocListDF_All.p'

MD_Doc = pd.read_pickle(path)

In [ ]:
len(MD_Doc['NPI'].unique())

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

# db_connection_str = 'mysql+pymysql://root:@localhost:3306/doctorinfo_sample?charset=utf8'
# db_connection = create_engine(db_connection_str)

# df = pd.read_sql('SELECT * FROM physicians_sample', con=db_connection)


# df.to_pickle('DocListDF.p')




def get_doctor_info(DoctorDF):


    doctor_infolist = []
    for idx, row in DoctorDF.iterrows():

        # most results: fistname is before lastname
        if row['MiddleName'] == None:
            doctor_info = "{} {} {} {} {} {}".format(row['FirstName'], row['LastName'], 
                                                  row['Primaryspecialty'], 
                                                  row['City'], row['State'],
                                                  row['Zip.Code'])
        else:
            doctor_info = "{} {} {} {} {} {} {}".format(row['FirstName'], row['MiddleName'],  row['LastName'], 
                                                  row['Primaryspecialty'], 
                                                  row['City'], row['State'],
                                                  row['Zip.Code'])
            
            
        # print(doctor_info)
        doctor_infolist.append([row['NPI'], doctor_info])

    return doctor_infolist






DocListDF_path = 'DocListDF.p'

df = pd.read_pickle(DocListDF_path) 


cols = ['cId', 'crawlFlag', 'NPI',  'LastName', 'FirstName',
       'MiddleName',  'City', 'State', 'Zip.Code', 'Primaryspecialty']

DoctorDF = df[cols]



doctor_infolist = get_doctor_info(DoctorDF)


len(doctor_infolist)



start = 0
length = 500

end = start + length

doctor_infolist = doctor_infolist[start: end]
len(doctor_infolist)

In [ ]:
def get_doctor_info(DoctorDF):
    doctor_infolist = []
    for idx, row in DoctorDF.iterrows():

        # most results: fistname is before lastname
        if row['MiddleName'] == None:
            doctor_info = "{} {} {} {} {} {}".format(row['FirstName'], row['LastName'], 
                                                  row['Primaryspecialty'], 
                                                  row['City'], row['State'],
                                                  row['Zip.Code'])
        else:
            doctor_info = "{} {} {} {} {} {} {}".format(row['FirstName'], row['MiddleName'],  row['LastName'], 
                                                  row['Primaryspecialty'], 
                                                  row['City'], row['State'],
                                                  row['Zip.Code'])
            
            
        # print(doctor_info)
        doctor_infolist.append([row['NPI'], doctor_info])

    return doctor_infolist



doctor_infolist = get_doctor_info(DoctorDF)



doctor_infolist[:10]


# Google Search URL

In [ ]:

import pandas as pd
# from bs4 import BeautifulSoup
import re

import requests
from scrapy.http import TextResponse

import cloudscraper







# we need headers to disguise our bot as a browser

HEADERS = {
    "Connection": "keep-alive",
    "Cache-Control": "max-age=0",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    # "User-Agent": "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/34.0.1847.131 Safari/537.36",
    'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36',
    "Accept-Encoding": "gzip,deflate,sdch",
    # "Accept-Language": "zh-CN,zh;q=0.8,en-US;q=0.6,en;q=0.4,zh-TW;q=0.2",
}

HEADERS = {'User-Agent': 'Mozilla/5.0 (X11; CrOS x86_64 12871.102.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.141 Safari/537.36',}

scraper = cloudscraper.create_scraper()

for doctor_info in doctor_infolist:
    NPI, keyword = doctor_info
    # break

    # print(keyword, NPI)
    
    
page_number = 1
print(NPI, keyword)

In [ ]:
   
# r = requests.get(url, headers=HEADERS, timeout=10)
# r.url


# load the text to scrapy-type response
# response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

# have a look at its body
# print(str(response.body.decode()))


# with open('preview.html', 'w') as f:
#     f.write(str(response.body.decode()))
    
# d = response.selector.xpath('.//div[@class="g"]//div[@data-hveid]//a/@href').extract()
# d = [i for i in d if 'http' in i]
# print(len(d))

In [ ]:
from pprint import pprint

def get_doctor_google_result(keyword, pages = 2, timeout = 10):
    
    searched_urls = []
    
    # first two pages
    for page_number in range(pages):
        url = "https://www.google.com/search?q=%s&start=%s" % (keyword, (page_number)*10)
        # print(url)
        r = requests.get(url, headers=HEADERS, timeout=timeout)
        
        # try:
        #     r = requests.get(url, headers=HEADERS, timeout=10)
        # except:
        #     r = scraper.get(url, headers=HEADERS, timeout=10)
            
            
        response = TextResponse(r.url, body = r.text, encoding = 'utf-8')

        # with open('preview.html', 'w') as f:
        #     f.write(str(response.body.decode()))
        
        
        d = response.selector.xpath('.//div[@class="g"]//div[@data-hveid]//a/@href').extract()
        # pprint(d)
        d = [i for i in d if 'http' in i and 'googleusercontent' not in i and 'translate.google.com' not in i]
        
        print('From ULR {}'.format(url))
        print('Get {} results'.format(len(d)))
        searched_urls = searched_urls + d
        
    return searched_urls

searched_urls = get_doctor_google_result(keyword, pages = 2)
searched_urls

In [ ]:
import pandas 

# Loop All

In [ ]:
from datetime import datetime
import numpy as np
import time
import os


tmp_path = 'DocGoogleURL.p' # df.to_pickle()


if os.path.exists(tmp_path):
    collected_NPIs = pd.read_pickle(tmp_path)['NPI'].to_list()
else:
    collected_NPIs = []
    
print('Collected NPI {}'.format(len(collected_NPIs)))


L = []
for doctor_info in doctor_infolist:
    
    NPI, keyword = doctor_info
    
    if NPI in collected_NPIs:
        continue
        
    
    print('\n\n', str(datetime.now()), keyword, NPI)
    
    
    try: 
        searched_urls = get_doctor_google_result(keyword, pages = 2)
        status = 1
        timestamp = str(datetime.now())
        
        
        second = random.randrange(1, 5)
        # time.sleep(second)
    
    except:
        print('No result for:', NPI, keyword)
        searched_urls = []
        status = 0
        timestamp = np.nan
    
    d = {
        'NPI': NPI, 
        'status': status,
        'searched_urls': searched_urls,
        'clct_time': timestamp
    }
    
    
    L.append(d)
    
    pd.DataFrame(L).df.to_pickle('DocGoogleURL.p')
    
    
    

In [ ]:
df.to_pickle('DocGoogleURL.p')